This script performs the Kolmogorov-Smirnov test for invariance on the
time series of microprice, expressed as a function of both clock time and
volume time. The results are then plotted and compared within a single
figure.
-

## For details, see [here](https://www.arpm.co/lab/redirect.php?permalink=iidhfmicroprice-copy-1).

In [ ]:
# ## Prepare the environment

In [ ]:
import os
import os.path as path
import sys

sys.path.append(path.abspath('../../functions-legacy'))

import numpy as np
from numpy import arange, interp, floor, diff

from scipy.io import loadmat

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

plt.style.use('seaborn')

from CONFIG import GLOBAL_DB, TEMPORARY_DB
from ARPM_utils import save_plot
from TestKolSmirn import TestKolSmirn
from InvarianceTestKolSmirn import InvarianceTestKolSmirn

In [ ]:
# ## Load the database generated by the script S_HighFreqVolumeTime
try:
    db = loadmat(os.path.join(GLOBAL_DB, 'db_HighFreqVolumeTime'))  # generated by S_HighFreqVolumime
except:
    db = loadmat(os.path.join(TEMPORARY_DB, 'db_HighFreqVolumeTime'))  # generated by S_HighFreqVolumime
p_mic = db['p_mic']
t_ms = db['t_ms']
t_k = db['t_k']
q_t = db['q_t']
q = db['q']

In [ ]:
# ## Compute microprice time series

delta_t = 2000  # selecting observations every 2 seconds
p_mic_t = p_mic[0, ::delta_t]

delta_q = floor((np.nanmax(q_t) - np.nanmin(q_t)) / len(p_mic_t))  # width of activity time bins
volume_time = arange(np.nanmin(q_t), np.nanmax(q_t) + delta_q, delta_q)  # vector of volume times
t_q = interp(volume_time, q[0], t_k[0])  # vector of wall clock time as a function of volume time
p_mic_q = interp(t_q, t_ms[0], p_mic[0])  # time changed microprice, i.e. microprice as a function of volume time

## Compute microprice increments

In [ ]:
delta_p_mic_t = diff(p_mic_t).reshape(1, -1)
delta_p_mic_q = diff(p_mic_q).reshape(1, -1)

## Compute the Kolmogorov-Smirnov test for microprice increments

In [ ]:
s1_t, s2_t, int_t, F1_t, F2_t, up_t, low_t = TestKolSmirn(delta_p_mic_t)
s1_q, s2_q, int_q, F1_q, F2_q, up_q, low_q = TestKolSmirn(delta_p_mic_q)

## Plot the results of the IID test

In [ ]:
# position settings
pos = {}
pos[0] = [0.1300, 0.74, 0.3347, 0.1717]
pos[1] = [0.5703, 0.74, 0.3347, 0.1717]
pos[2] = [0.1300, 0.11, 0.7750, 0.5]
pos[3] = [0.05, 1.71]

f = figure()
InvarianceTestKolSmirn(delta_p_mic_t, s1_t, s2_t, int_t, F1_t, F2_t, up_t, low_t, pos,
                       'Test on microprice as a function of clock-time');
# save_plot(ax=plt.gca(), extension='png', scriptname=os.path.basename('.')[:-3], count=plt.get_fignums()[-1])

f = figure()
InvarianceTestKolSmirn(delta_p_mic_q, s1_q, s2_q, int_q, F1_q, F2_q, up_q, low_q, pos,
                       'Test on microprice as a function of volume activity-time');
# save_plot(ax=plt.gca(), extension='png', scriptname=os.path.basename('.')[:-3], count=plt.get_fignums()[-1])